In [12]:
import pandas as pd
import yake
import re
import threading
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from collections import Counter

In [13]:
# Importing dataset, enter the path of the csv file in the csv_file_name variable
csv_file_name="dataset/train.csv"
sum_df = pd.read_csv(csv_file_name)
sum_df['NUMERIC_DATA'] = ''
sum_df=sum_df.dropna()
# sum_df=sum_df.sample(n=100000)

In [23]:
# def extract_keywords(text):
def getKeywordsUsingYake(text):

    # Create a YAKE keyword extractor
    kw_extractor = yake.KeywordExtractor(lan="en", n=1, dedupLim=0.9, top=20)

    # Extract keywords from the text
    keywords = kw_extractor.extract_keywords(text)

    # Join keywords into a comma-separated string
    keyword_string = ", ".join([keyword[0] for keyword in keywords])

    return keyword_string

In [3]:
# def getKeywordsUsingRake(text, stop_words):
    # # Tokenize text and remove stop words
    # tokens = word_tokenize(text)
    # # stop_words = set(stopwords.words('english'))
    # filtered_tokens = [token for token in tokens if token.lower() not in stop_words]

    # # Stem tokens
    # stemmer = PorterStemmer()
    # stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]

    # # Calculate word scores using RAKE algorithm
    # word_scores = {}
    # for token in stemmed_tokens:
    #     if token not in word_scores:
    #         word_scores[token] = 0
    #     word_scores[token] += 1

    # for token in word_scores:
    #     word_scores[token] /= float(len(stemmed_tokens))

    # # Sort words by score and extract top 20
    # sorted_words = sorted(word_scores.items(), key=lambda x: x[1], reverse=True)
    # keywords = [word[0] for word in sorted_words[:20]]

    # #removing the special character, emojis, and punctuation

    # opstr = ', '.join(keywords)
    # return opstr


In [14]:
def getKeywordsUsingRake(text, stop_words):
    # Tokenize text and remove stop words
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]

    # Calculate word scores using RAKE algorithm
    word_scores = {}
    for token in filtered_tokens:
        if token not in word_scores:
            word_scores[token] = 0
        word_scores[token] += 1

    for token in word_scores:
        word_scores[token] /= float(len(filtered_tokens))

    # Sort words by score and extract top 20
    sorted_words = sorted(word_scores.items(), key=lambda x: x[1], reverse=True)

    # Remove stopwords and special characters
    new_list = []

    # Loop through each item in the original list
    for item in sorted_words:
        # Check if the item contains only alphanumeric characters
        if len(item[0])==1:
            continue
        if item[0].isalnum():
            # If the item contains only alphanumeric characters, add it to the new list
            new_list.append(item[0])
    keywords = [word for word in new_list[:30]]
    opstr = ', '.join(keywords)
    return opstr


In [4]:
def numeric_data_extractor(description):
    

    length_pattern = r"\b(?:\d+(?:\s\d+/\d+)?(?:\.\d+)?|\d+/\d+)(?:\s*\*\s*(?:\d+(?:\s\d+/\d+)?(?:\.\d+)?|\d+/\d+))*\s*(?:cm|mm|m|in(?:ch(?:es)?)?|ft|foot|feet|yd|yard|')?\b"


    lengths = re.findall(length_pattern, description, re.IGNORECASE)

    

    opstr=""
    for i in lengths:
        opstr=opstr+i+","
    
    return opstr

In [15]:
stop_words = set(stopwords.words('english'))

In [6]:
# Set the column name
column_name_1 = "TITLE"
column_name_2 = "BULLET_POINTS"
column_name_3 = "DESCRIPTION"

# Loop through each row in the dataframe
for index, row in sum_df.iterrows():
    # Extract the text to be summarized from the column
    text_1 = row[column_name_1]
    text_2 = row[column_name_2]
    text_3 = row[column_name_3]

    inpstr= text_1+" "+text_2+" "+text_3
    
    # Use get numeric data
    ndata=numeric_data_extractor(inpstr)

    # Replace the original text with the summary in the same column
    sum_df.at[index, 'NUMERIC_DATA'] = ndata

In [16]:
# Define a function that runs code block 1
def process_column_1():
    # Set the column name
    column_name = "BULLET_POINTS"

    # Loop through each row in the dataframe
    for index, row in sum_df.iterrows():
        # Extract the text to be summarized from the column
        text = row[column_name]

        # Use yake to extract 20 keywords 
        kw = getKeywordsUsingRake(text,stop_words)

        # Replace the original text with the summary in the same column
        sum_df.at[index, column_name] = kw

In [17]:
# Define a function that runs code block 2
def process_column_2():
    # Set the column name
    column_name = "DESCRIPTION"

    # Loop through each row in the dataframe
    for index, row in sum_df.iterrows():
        # Extract the text to be summarized from the column
        text = row[column_name]

        # Use yake to extract 20 keywords 
        kw = getKeywordsUsingRake(text,stop_words)

        # Replace the original text with the summary in the same column
        sum_df.at[index, column_name] = kw

In [18]:

# Manages threads so that the first and second code block run simultaneously


# Create two threads, one for each code block
t1 = threading.Thread(target=process_column_1)
t2 = threading.Thread(target=process_column_2)

# Start both threads
t1.start()
t2.start()

# Wait for both threads to finish
t1.join()
t2.join()

In [19]:
sum_df.head()

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH,NUMERIC_DATA
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"Compatible, SX4, Horn, High, Loud, Dual, Tone,...","Voltage, Material, 12V, dB, Pump, Dual, Tone, ...",7537,748.031495,
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,"fabric, Made, 95, cotton, Lycra, gives, ways, ...","inch, Ankel, Leggings, Length, 30, 32, 34, col...",2996,787.401574,
5,2152929,HINS Metal Bucket Shape Plant Pot for Indoor &...,"plants, indoor, stand, plant, Simple, elegant,...","HINS, Pot, one, choice, plants, take, color, r...",5725,950.000000,
7,2026580,Delavala Self Adhesive Kitchen Backsplash Wall...,"kitchen, PVC, aluminum, foil, made, plastic, m...","br, nbsp, strong, kitchen, Aluminum, Foil, tim...",6030,984.251967,
9,2998633,Hexwell Essential oil for Home Fragrance Oil A...,"Oil, essential, Essential, oils, oil, could, u...","oils, Transform, home, workplace, hotel, room,...",8201,393.700787,


In [11]:
# saving the dataframe
sum_df.to_csv("dataset/updated_dataset.csv")